# Overview

#### Objective: Move the CSVs into a denormalized Apache Cassandra database that we can run queries on.

##### Steps
###### Step 1: Process the CSVs into a single data file
###### Step 2: Create Apache Cassandra cluster and keyspace.
###### Step 3: Model the database.
###### Step 4: Build ETL pipeline.
###### Step 5: 

# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(f"Current working directory : {os.getcwd()}") # Added f string, Current working directory to make steps more clear

# Create a filepath variable composed of your current folder/working directory and subfolder event data
# Note that os.walk returns a generator, that creates a tuple of values (current_path, \
# directories in current_path, files in current_path)
filepath = os.getcwd() + '/event_data'

# Create a for loop using os.walk to literally walk through that directory structure to \
# 1 - create a list of files/tuple of values and 2 - collect each filepath for eventual integration
# A root is the highest directory in the filepath, e.g. C:\
# A directory is the folder holding the file, e.g. C:\kgwhite2\Documents\Taxes
# A file is the resource for recording data, e.g. C:\kgwhite2\Documents\Taxes\2020return.pdf
for root, dirs, files in os.walk(filepath):
    
# join the file paths (files) and roots (root) into 1 variable with the subdirectories (dir) using glob
# glob is a Python module that's useful whenever we need to look for a list of files with names matching a pattern
# With glob, I can scan a directory for any file that has a certain extension, prefix or comon string in the middle
    file_path_list = glob.glob(os.path.join(root,'*')) # glob creates a list of filepaths to process all event_data
    print(file_path_list)

Current working directory : /home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/ev

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# Create/Initiate an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# loop through every filepath in the filepath list (created in the glob step)
for f in file_path_list:

# Read each csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
# Extract data from each row one by one and append it to the empty list of rows \
# (created in the full_data_rows_list =[] step)       
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# The code below gets the total number of rows. I added an F string to form a sentence with it. 
print(f"Total rows : {len(full_data_rows_list)}")
# The code below gets what the list of event data rows will look like. I added an F string to form a sentence with it. 
print(f"Sample data:\n {full_data_rows_list[:5]}")


# I will now create a smaller event data csv file called event_datafile_full csv that \
# will be used to insert data into the Apache Cassandra tables
# csv.register_dialect creates a new dialect that specifies the CSV file's delimiter and other parameters
# csv.register_dialect(name, formattingparameters) is the common format
# It can be erased with csv.unregister_dialect(name)
# We name our new dialect 'myDialect'
# The quoting=csv.QUOTE_ALL formatting parameter quotes (includes?) everything, regardless of type
# The skipinitialspace=True formatting parameter controls how the space following the delimiter will be interpreted. 
# E.g. If True, the initial whitespaces will be removed. In ["Adam Donachie", "BAL"], the whitespace is ignored. False is the default value.
# For more learning, look into Python CSV package tutorials on thepythonguru.com
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)


with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    #using the dialect created above, create the smaller event_data csv file
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

# I've now completed Step 1 and processed these data into a single data file /
# to move into the Apache Cassandra database

Total rows : 8056
Sample data:
 [['Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner', 'Logged In', 'Mohammad', 'M', '0', 'Rodriguez', '277.15873', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)', '200', '1.54328E+12', '88'], ['Jimi Hendrix', 'Logged In', 'Mohammad', 'M', '1', 'Rodriguez', '239.82975', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Woodstock Inprovisation', '200', '1.54328E+12', '88'], ['Building 429', 'Logged In', 'Mohammad', 'M', '2', 'Rodriguez', '300.61669', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Majesty (LP Version)', '200', '1.54328E+12', '88'], ["The B-52's", 'Logged In', 'Gianna', 'F', '0', 'Jones', '321.54077', 'free', 'New York-Newark-Jersey City, NY-NJ-PA', 'PUT', 'NextSong', '1.54087E+12', '107', 'Love Shack', '200', '1.54328E

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
    print("Connection Established !!")
except Exception as e:
    print(f"Connection Failed !! Error : {e}")

Connection Established !!


#### Create Keyspace
##### In a relational data model, this would be a database. In Cassandra, databases are known as keyspaces.

In [6]:
# TO-DO: Create a Keyspace 

# Create the sparkify keyspace
keyspace_query = """CREATE KEYSPACE IF NOT EXISTS sparkify 
                    with REPLICATION = 
                    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
                """

try:
    session.execute(keyspace_query)
except Exception as e:
    print(f"Failed to create keyspace!! Error : {e}")

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above

session.set_keyspace('sparkify')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




====================================================================================================================================

## Query 1

#### Remember, with Apache Cassandra, we model the database tables on the queries we want to run!
#### For Query 1, we'll A) create a table and B) run a query that selects the 1) artist name, 2) song name, and 3) song length from the table, while filtering by 4) sessionId and 5) itemInSession.

   > * Create table with 5 columns of 1) artist name, 2) song name, and 3) song length 4) sessionId and 5) itemInSession from the single data file we created above
   > * Run SQL query filtering the table by sessionId = 338 and itemInSession = 4

In [8]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

## We 1) create the table that the query runs off of, 2) then we run the query
## 1) Create the Table
## Include: 1) naming the table (session_ken), 2) defining the columns with data type, 3) setting a primary key, \
## and 4) setting a clustering key
## The primary key is known as a partition key in Cassandra. The primary key is a special relational database table \
## column (or combination of columns) designated to uniquely identify each table record.
## The Cassandra clustering keys are the other fields we'll use to filter results by. This approach is unique to \
## Cassandra and is tied to how in Cassandra, we must model the database tables on the queries we want to run!

createtable_query1 = """CREATE TABLE IF NOT EXISTS session_ken (artist text, song text, length float, sessionId int,
itemInSession int, PRIMARY KEY (sessionId, itemInSession))"""

try: 
    session.execute(createtable_query1)
    print("Table Created!!")
except Exception as e:
    print(f"Table creation failed!! Error : {e}")

Table Created!!


In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
# Define the file we want to use for uploading data into the 'session_ken' columnfamily, which is called a table \
# in a relational data model
file = 'event_datafile_new.csv'

# We'll read the CSV event file and insert rows into the session_ken table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
## ?
        query = "INSERT INTO session_ken (artist, song, length, sessionId, itemInSession)" 
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], line[10], float(line[5]), int(line[8]), int(line[3])) )

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
## ?

select_query1 = "SELECT artist, song, length FROM  session_ken where sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(select_query1)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist='Faithless', song='50', length=495.30731201171875)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [11]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

## ?
create_query2 = """CREATE TABLE IF NOT EXISTS user_session (sessionId int, userId int, artist text, 
song text, firstName text, lastName text, itemInSession int, PRIMARY KEY ((sessionId, userId), 
itemInSession)) WITH CLUSTERING ORDER BY (itemInSession ASC) """

try: 
    session.execute(create_query2)
    print("Table Created!!")
except Exception as e:
    print(f"Table creation failed!! Error : {e}")
                    

Table Created!!


In [12]:
## ?

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session (sessionId, userId, artist, song, firstName, lastName, itemInSession) "
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s) "
        session.execute(query, (int(line[8]), int(line[10]), line[0], line[9], line[1], line[4], int(line[3])  ) )

In [13]:
# SELECT statement to verify the data was entered into the table
## ?

select_query2 = "SELECT artist, song, firstName, lastName FROM  user_session where sessionId = 182 and userId = 10"
try:
    rows = session.execute(select_query2)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


In [14]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
## ?

create_query3 = """CREATE TABLE IF NOT EXISTS user_song_ken (song text, userId int, firstName text, lastName text, PRIMARY KEY ((song), userId))"""

try: 
    session.execute(create_query3)
    print("Table Created!!")
except Exception as e:
    print(f"Table creation failed!! Error : {e}")

Table Created!!


In [15]:
## ?

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_song_ken (song, userId, firstName, lastName) "
        query = query + " VALUES (%s, %s, %s, %s) "
        session.execute(query, (  line[9], int(line[10]), line[1], line[4] )  )

In [16]:
# SELECT statement to verify the data was entered into the table
## SEE OTHER NOTES!

select_query3 = "SELECT song, firstName, lastName FROM user_song_ken where song = 'All Hands Against His Own'"
try:
    rows = session.execute(select_query3)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(song='All Hands Against His Own', firstname='Jacqueline', lastname='Lynch')
Row(song='All Hands Against His Own', firstname='Tegan', lastname='Levine')
Row(song='All Hands Against His Own', firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [17]:
## TO-DO: Drop the table before closing out the sessions
session.execute("DROP TABLE IF EXISTS sparkify.session_ken")
session.execute("DROP TABLE IF EXISTS sparkify.user_session")
session.execute("DROP TABLE IF EXISTS sparkify.user_song_ken")

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()